In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import matplotlib.pyplot as plt
from datetime import datetime

df = pd.read_csv("../../../../Downloads/reviews.csv")
def normalize_rating(r):
    if pd.isnull(r):
        return np.nan

    r = r.strip().upper()

    #se è frazionario prende in base alla scala e moltiplica per 5, normalizzazione su 5 come per il sito
    fraction_match = re.match(r'(\d*\.?\d+)\s*/\s*(\d+)', r)
    if fraction_match:
        score = float(fraction_match.group(1))
        scale = float(fraction_match.group(2))
        return (score / scale) * 5

    try:   #puo essere direttamente numero in alcune
        val = float(r)
        return val if 0 <= val <= 5 else np.nan
    except:
        pass

    letter_grades = {
        'A+': 5.0, 'A': 5.0, 'A-': 4.5,
        'B+': 4.0, 'B': 3.5, 'B-': 3.0,
        'C+': 2.5, 'C': 2.0, 'C-': 1.5,
        'D+': 1.0, 'D': 1.0, 'D-': 0.5,
        'F': 0.0}
    return letter_grades.get(r, np.nan)

df['rating_normalized'] = df['rating'].apply(normalize_rating)

df['createdAt'] = pd.to_datetime(df['createdAt'], errors='coerce') #convnersione della data per l'anno

df = df[df['createdAt'].dt.year >= 2000] #filtro dopo ikl 2k, prima sono poche recensioni, per non specificare ogni grafico di prendere dal 2k 

df = df.dropna(subset=['rating_normalized', 'testata', 'content']) #rimozione righe nan o non normalizzate

df.drop(columns=['data_inu', 'data_inuu', 'link', '_id', 'imported', 'rating'], inplace=True) #colonne inutili per questi grafici

df['year'] = df['createdAt'].dt.year #colonna anno per brevità nell usarla


In [2]:
# Media dei rating per anno e categoria (solo Fresh/Rotten)

grouped = df.groupby(['year', 'fresh'])['rating_normalized'].mean().reset_index()

fig = px.line(
    grouped, x='year', y='rating_normalized', color='fresh',
    title="Average Rating per Anno per Categoria (Fresh vs Rotten)",
    template='plotly_dark', color_discrete_sequence=px.colors.sequential.Rainbow
)

fig.update_layout(xaxis=dict(title='Anno', dtick=1), yaxis_title="Average Rating (/5)")
fig.show()

df['period'] = (df['year'] // 2) * 2

#calcolo delle top 15 testate per numero di recensioni
top_testate = df['testata'].value_counts().nlargest(15).index.tolist()
filtered_df = df[df['testata'].isin(top_testate)]

grouped_2y = filtered_df.groupby(['testata', 'period'])['rating_normalized'].mean().reset_index()

fig = px.line(
    grouped_2y,
    x="period", y="rating_normalized", color="testata",
    title="Media Rating per Testata (Top 15) Ogni 2 Anni",
    template="plotly_dark", color_discrete_sequence=px.colors.sequential.Rainbow,
    height=700
)

fig.update_layout(xaxis=dict(title="Periodo (ogni 2 anni)", dtick=2), yaxis_title="Media Rating (/5)")
fig.show()
#le reviews del newyork times si fermano al 2013 anche nel csv

In [3]:
# Top 10 testate per numero di recensioni

top_outlets = df['testata'].value_counts().nlargest(10).index.tolist() #top 10 critiche(per numero di critiche)
df_top_outlets = df[df['testata'].isin(top_outlets)]

fig = px.box(
    df_top_outlets,
    x="testata", y="rating_normalized", color="fresh",
    title="Distribuzione Rating per Testate (Boxplot - Fresh vs Rotten)",
    template="plotly_dark", color_discrete_sequence=px.colors.sequential.Rainbow
)

fig.update_layout(xaxis_title="Testata Giornalistica", yaxis_title="Rating Normalizzato (/5)")
fig.show()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

fresh_reviews = df[df['fresh'] == 'Fresh']['content'].dropna().tolist()   #analizzo solo il content filtrandolo con stopword inglesi generali + aggiunte date dai primi risultati
rotten_reviews = df[df['fresh'] == 'Rotten']['content'].dropna().tolist()
all_reviews = fresh_reviews + rotten_reviews

custom_stopwords = list(ENGLISH_STOP_WORDS.union({
    'movie', 'film', 'director', 'screenplay', 'screen', 'cinema',
    'watch', 'viewing', 'role', 'films', 'movies', 'play',
    'doesn', 'isn', 'like', 'way', 'really'
}))

# Vectorizer
vectorizer = CountVectorizer(stop_words=custom_stopwords, min_df=5)
X_all = vectorizer.fit_transform(all_reviews)


X_fresh = X_all[:len(fresh_reviews), :]
X_rotten = X_all[len(fresh_reviews):, :]


vocab = vectorizer.get_feature_names_out()
freq_fresh = pd.DataFrame(X_fresh.toarray(), columns=vocab).sum().reset_index()    #la serie viene trasformata in un dataframe a due colonne parole/somma comparse
freq_rotten = pd.DataFrame(X_rotten.toarray(), columns=vocab).sum().reset_index()
freq_fresh.columns = ['word', 'count_fresh']
freq_rotten.columns = ['word', 'count_rotten']


merged = pd.merge(freq_fresh, freq_rotten, on='word', how='outer').fillna(0)
merged['total'] = merged['count_fresh'] + merged['count_rotten']
top_words = merged.sort_values(by='total', ascending=False).head(25)


melted = top_words.melt(id_vars='word', value_vars=['count_fresh', 'count_rotten'],
                        var_name='tipo', value_name='frequenza')


fig = px.bar(
    melted, x='word', y='frequenza', color='tipo',
    barmode='group', title='Top 25 Parole Più Frequenti: Fresh vs Rotten',
    template='plotly_dark', color_discrete_sequence=px.colors.sequential.Rainbow
)
fig.update_layout(xaxis_title='Parola', yaxis_title='Frequenza')
fig.show()


"{sys.executable}" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.
